# Importing the data and Libraries

In [4]:
from sklearn.datasets import load_iris
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings(action="ignore")
sns.set()

In [10]:
data=pd.read_csv("iris.csv")

In [22]:
x,y=data.drop("class",axis=1),data.iloc[:,-1]

In [11]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [31]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(metric="wminkowski",metric_params={'w':1},p=1)
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'wminkowski',
 'metric_params': {'w': 1},
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 1,
 'weights': 'uniform'}

## Parameters
#---Algorithm----
* ‘ball_tree’ will use BallTree
* ‘kd_tree’ will use KDTree
* ‘brute’ will use a brute-force search.
* ‘auto’ will attempt to decide the most appropriate algorithm based on the values passed to fit method.

#---leaf_size,p,n_neighnors --integer

#---Weights
* uniform
* distance

![](MetricMeasure.png)

In [28]:
knn.fit(x,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='chebyshev',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')